# NFL Fantasy Football Projection Model Using XGBoost

This notebook presents a comprehensive workflow for building an NFL fantasy football projection model using a Bayesian Method. Data sources include Yahoo, Pro-Football Reference, SportsDataIO, and NFLVerse. The goal is to leverage advanced machine learning techniques and rich datasets to generate accurate player projections for fantasy football analysis and decision-making.

In [1]:
import pandas as pd
import numpy as np
import os
import sys

project_root = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
print(f"Project Root: {project_root}")
print("Sys Path Before:", sys.path)
if project_root not in sys.path:
    print("Inserting project root to sys.path")
    sys.path.insert(0, project_root)

# Now import
from data_api import SportsDataIO, Yahoo, PFR, POSITION_PLAYER_STAT_PROJECTION_DATA_DICT
import utils
import yahoo_helpers

from dotenv import load_dotenv
load_dotenv()

Project Root: c:\Users\bengu\Documents\NFL Data Project\clairvoyent-raven-sports-analysis\src
Sys Path Before: ['C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\python310.zip', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\DLLs', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310\\lib', 'C:\\Users\\bengu\\AppData\\Local\\Programs\\Python\\Python310', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj', '', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\win32\\lib', 'c:\\Users\\bengu\\.virtualenvs\\cfeproj-oIABPDjj\\lib\\site-packages\\Pythonwin']
Inserting project root to sys.path


True

## Import and clean data from nflverse source

In [2]:
NFLVERSE_DATA_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_player_stats_cleaned.csv"
NFLVERSE_TEAMS_DATA_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_team_stats.xlsx"
NFLVERSE_INJURIES_PATH = r"C:\Users\bengu\Documents\Sports Analysis Project\clairvoyent-raven-sports-analysis\data\nfl_injuries.xlsx"
NFLVERSE_DEPTH_CHART_PATH = r"C:\Users\bengu\Documents\NFL Data Project\clairvoyent-raven-sports-analysis\data\nfl_depth_charts.xlsx"

all_players_df = pd.read_csv(NFLVERSE_DATA_PATH)
all_teams_df = pd.read_excel(NFLVERSE_TEAMS_DATA_PATH, engine="openpyxl")
injuries_df = pd.read_excel(NFLVERSE_INJURIES_PATH, engine="openpyxl")
depth_df = pd.read_excel(NFLVERSE_DEPTH_CHART_PATH, engine="openpyxl")

In [3]:
from sklearn.preprocessing import OneHotEncoder

if "gsis_id" in injuries_df.columns:
   injuries_df = injuries_df.rename({"gsis_id": "player_id"}, axis=1)

filtered_injuries_df = injuries_df[["season", "week", "player_id", "report_status", "practice_status"]]

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

X_categorical_injury = filtered_injuries_df[["report_status", "practice_status"]]

encoded_data = encoder.fit_transform(X_categorical_injury)

encoded_feature_names = encoder.get_feature_names_out(X_categorical_injury.columns)

encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names)

filtered_injuries_df = pd.concat([filtered_injuries_df[["season", "week", "player_id"]], encoded_df], axis=1)

In [4]:
if "gsis_id" in depth_df.columns:
    filtered_depth_df = depth_df.rename({"gsis_id": "player_id"}, axis=1)

filtered_depth_df = filtered_depth_df[["season", "week", "player_id", "depth_team"]]
filtered_depth_df = filtered_depth_df.dropna(subset=["season", "week"])
filtered_depth_df["week"] = filtered_depth_df["week"].astype(int)
filtered_depth_df["season"] = filtered_depth_df["season"].astype(int)


In [5]:
"""Merge depth chart information and injury information onto the all players data frame.
"""

all_players_df = all_players_df.merge(filtered_injuries_df, how="left", on=["player_id", "season", "week"])

all_players_df = all_players_df.merge(filtered_depth_df, how="left", on=["player_id", "season", "week"])


In [6]:
# Fill none values for depth chart with the average value.
# Fill the injury encoded values with 0 because it's safe to assume that no injury occurred

depth_mean = all_players_df["depth_team"].mean()
all_players_df["depth_team"] = all_players_df["depth_team"].fillna(depth_mean)
all_players_df[encoded_feature_names] = all_players_df[encoded_feature_names].fillna(0)

In [7]:
column_categories = {
    "standard": {
        "player_id",
        "player_name",
        "position",
        "position_group",
        "season",
        "week",
        "season_type",
        "team",
        "opponent_team",
        "pacr",
        "report_status_Doubtful",
        "report_status_Note",
        "report_status_Out",
        "report_status_Probable",
        "report_status_Questionable",
        "report_status_nan",
        "practice_status_\n    ",
        "practice_status_Did Not Participate In Practice",
        "practice_status_Full Participation in Practice",
        "practice_status_Limited Participation in Practice",
        "practice_status_Note",
        "practice_status_Out (Definitely Will Not Play)",
        "depth_team", 
    },
    "passing": {
        "completions",
        "attempts",
        "passing_yards",
        "passing_tds",
        "passing_interceptions",
        "sacks_suffered",
        "sack_yards_lost",
        "sack_fumbles",
        "sack_fumbles_lost",
        "passing_air_yards",
        "passing_yards_after_catch",
        "passing_first_downs",
        "passing_epa",
        "passing_cpoe",
        "passing_2pt_conversions",
        "carries", # Include rushing stats for QBs
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
    },
    "rushing_and_receiving": {
        "carries",
        "rushing_yards",
        "rushing_tds",
        "rushing_fumbles",
        "rushing_fumbles_lost",
        "rushing_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "rushing_epa",
        "rushing_2pt_conversions",
        "receptions",
        "targets",
        "receiving_yards",
        "receiving_tds",
        "receiving_fumbles",
        "receiving_fumbles_lost",
        "receiving_air_yards",
        "receiving_yards_after_catch",
        "receiving_first_downs",
        "receiving_epa",
        "receiving_2pt_conversions",
        "racr",
        "target_share",
        "air_yards_share",
        "wopr",
    },
    "defense": {
        "special_teams_tds",
        "def_tackles_solo",
        "def_tackles_with_assist",
        "def_tackle_assists",
        "def_tackles_for_loss",
        "def_tackles_for_loss_yards",
        "def_fumbles_forced",
        "def_sacks",
        "def_sack_yards",
        "def_qb_hits",
        "def_interceptions",
        "def_interception_yards",
        "def_pass_defended",
        "def_tds",
        "def_fumbles",
        "def_safeties",
        "misc_yards",
        "fumble_recovery_own",
        "fumble_recovery_yards_own",
        "fumble_recovery_opp",
        "fumble_recovery_yards_opp",
        "fumble_recovery_tds",
        "penalties",
    },
    "kicking": {
        "fg_made",
        "fg_att",
        "fg_missed",
        "fg_blocked",
        "fg_long",
        "fg_pct",
        "fg_made_0_19",
        "fg_made_20_29",
        "fg_made_30_39",
        "fg_made_40_49",
        "fg_made_50_59",
        "fg_made_60_",
        "fg_missed_0_19",
        "fg_missed_20_29",
        "fg_missed_30_39",
        "fg_missed_40_49",
        "fg_missed_50_59",
        "fg_missed_60_",
        "fg_made_list",
        "fg_missed_list",
        "fg_blocked_list",
        "fg_made_distance",
        "fg_missed_distance",
        "fg_blocked_distance",
        "pat_made",
        "pat_att",
        "pat_missed",
        "pat_blocked",
        "pat_pct",
        "gwfg_made",
        "gwfg_att",
        "gwfg_missed",
        "gwfg_blocked",
        "gwfg_distance",
    },
    "special_teams": {
        "punt_returns",
        "punt_return_yards",
        "kickoff_returns",
        "kickoff_return_yards",
    },
}

categories_positions = {
    "passing": ["QB"],
    "rushing_and_receiving": ["RB", "WR"],
    # Not available "kicking": ["K"] 
}


In [8]:
# Establish dataframes for each positional group, filter by position value
def filter_by_positional_group(df, category_key_a, category_key_b="standard") -> pd.DataFrame:
    new_df = df.loc[:, list(column_categories[category_key_a] | column_categories[category_key_b])]
    new_df = new_df[new_df["position"].isin(categories_positions[category_key_a])]
    return new_df.reset_index(drop=True)

# Data of interest
passing_df = filter_by_positional_group(all_players_df, "passing")
rushing_and_receiving_df = filter_by_positional_group(all_players_df, "rushing_and_receiving")

# Save in case
defense_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["defense"])]
kicking_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["kicking"])]
specials_teams_df = all_players_df.loc[:, list(column_categories["standard"] | column_categories["special_teams"])]

# Give the dataframes each an alias
passing_df.alias = "passing_df"
rushing_and_receiving_df.alias = "rushing_and_receiving_df"
defense_df.alias = "defense_df"
kicking_df.alias = "kicking_df"
specials_teams_df.alias = "specials_teams_df"

In [9]:
for df in [passing_df, rushing_and_receiving_df, defense_df, kicking_df, specials_teams_df]:
    null_values = 0
    print(f"Checking {df.alias} for null values")
    null_counts = df.isna().sum()

    for column, count in zip(null_counts.index, null_counts):
        if count != 0:
            null_values += 1
            print(f"Column: {column} contains {count} null values.")
    
    print(f"{df.alias} contains {null_values} null values.")

df = None

Checking passing_df for null values
passing_df contains 0 null values.
Checking rushing_and_receiving_df for null values
rushing_and_receiving_df contains 0 null values.
Checking defense_df for null values
defense_df contains 0 null values.
Checking kicking_df for null values
kicking_df contains 0 null values.
Checking specials_teams_df for null values
specials_teams_df contains 0 null values.


### Implement and Apply Model

Implement multiple linear regression models to predict multiple statistics for rushers and receivers. 

* Define the targets: rushing_yards, rushing_tds, receiving_yards, receiving_tds

In [7]:
rushing_and_receiving_df.columns

Index(['pacr', 'player_id', 'opponent_team', 'receiving_epa',
       'rushing_fumbles', 'target_share', 'rushing_yards', 'carries',
       'rushing_fumbles_lost', 'position_group', 'team', 'racr',
       'receiving_yards', 'receptions', 'receiving_yards_after_catch',
       'receiving_tds', 'season', 'receiving_fumbles', 'wopr',
       'receiving_first_downs', 'rushing_2pt_conversions',
       'receiving_air_yards', 'week', 'rushing_epa', 'position',
       'air_yards_share', 'season_type', 'targets', 'rushing_first_downs',
       'rushing_tds', 'receiving_fumbles_lost', 'receiving_2pt_conversions',
       'player_name'],
      dtype='object')

In [10]:
"""
Collect xs, and ys.
Normalize data using Z-score normalization
(val - mean_val) / standard_deviation

Create 4 buckets, one for each statistical category.

Prefix   | Stat Category
-------------------------
rsh_yd_ -> rushing_yards
rsh_td_ -> rushing_tds
rc_yd_  -> receiving_yards
rc_td_  -> receiving_tds
"""

inputs = {
    "rsh_yd": ["rushing_tds", "rushing_yards", "rushing_fumbles_lost", "racr", "rushing_epa", "rushing_fumbles", "carries", "pacr", "rushing_first_downs", "rushing_2pt_conversions"],
    "rsh_td": ["rushing_tds", "rushing_yards", "rushing_fumbles_lost", "racr", "rushing_epa", "rushing_fumbles", "carries", "pacr", "rushing_first_downs", "rushing_2pt_conversions"],
    "rc_yd" : ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards", "receiving_fumbles_lost", "receptions"],
    "rc_td" : ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards", "receiving_fumbles_lost", "receptions"],
    "rc": ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards", "receiving_fumbles_lost", "receptions"],
    "p_yd": ["completions", "attempts", "passing_yards", "passing_tds", "passing_interceptions", "sacks_suffered", "sack_yards_lost", "sack_fumbles", "sack_fumbles_lost", "passing_air_yards", "passing_yards_after_catch", "passing_first_downs", "passing_epa", "passing_cpoe", "passing_2pt_conversions", "pacr"],
    "p_td": ["completions", "attempts", "passing_yards", "passing_tds", "passing_interceptions", "sacks_suffered", "sack_yards_lost", "sack_fumbles", "sack_fumbles_lost", "passing_air_yards", "passing_yards_after_catch", "passing_first_downs", "passing_epa", "passing_cpoe", "passing_2pt_conversions", "pacr"],
    "intcpt": ["completions", "attempts", "passing_yards", "passing_tds", "passing_interceptions", "sacks_suffered", "sack_yards_lost", "sack_fumbles", "sack_fumbles_lost", "passing_air_yards", "passing_yards_after_catch", "passing_first_downs", "passing_epa", "passing_cpoe", "passing_2pt_conversions", "pacr"],
    "rsh_fmbls": ["rushing_tds", "rushing_yards", "rushing_fumbles_lost", "racr", "rushing_epa", "rushing_fumbles", "carries", "pacr", "rushing_first_downs", "rushing_2pt_conversions"],
    "rc_fmbls": ["receiving_epa", "receiving_2pt_conversions", "racr", "air_yards_share", "receiving_tds", "receiving_first_downs", "wopr", "receiving_yards_after_catch", "receiving_air_yards", "target_share", "pacr", "targets", "receiving_yards", "receiving_fumbles_lost", "receptions"],
    "def": ["def_tackles_solo", "def_tackles_with_assist", "def_tackle_assists", "def_tackles_for_loss", "def_tackles_for_loss_yards", "def_fumbles_forced", "def_sacks", "def_sack_yards", "def_qb_hits", "def_interceptions", "def_interception_yards", "def_pass_defended", "def_tds", "def_fumbles", "def_safeties"]
}

# Stat 1 rushing_yards
rsh_yd_X = rushing_and_receiving_df[inputs["rsh_yd"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 2 rushing_tds
rsh_td_X = rushing_and_receiving_df[inputs["rsh_td"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 3 receiving_yards
rc_yd_X = rushing_and_receiving_df[inputs["rc_yd"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 4 receiving_tds
rc_td_X = rushing_and_receiving_df[inputs["rc_td"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 5 receptions
rc_X = rushing_and_receiving_df[inputs["rc"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 6 passing_yards
p_yd_X = passing_df[inputs["p_yd"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 7 passing_tds
p_td_X = passing_df[inputs["p_td"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 8 interceptions
intcpt_X = passing_df[inputs["intcpt"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
# Stat 9-10 fmbls
rsh_fmbls_X = pd.concat([rushing_and_receiving_df, passing_df]).reset_index(drop=True)[inputs["rsh_fmbls"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]
rc_fmbls_X = pd.concat([rushing_and_receiving_df, passing_df]).reset_index(drop=True)[inputs["rc_fmbls"] + ["season", "week", "player_id", "player_name", "opponent_team", "depth_team"] + list(encoded_feature_names)]

# Defensive stats for opponent
defense_stats_df = all_teams_df[inputs["def"] + ["season", "week", "team"]]

Calculate rolling data over a period of 3 weeks

In [11]:
def calculate_rolling_data(df: pd.DataFrame, sort_values: list, input_ref: str, groupby: list, rolling_period: int=3, min_periods: int=1, shift: int=1):
    df = df.sort_values(sort_values)

    df[[f"{c}_roll{rolling_period}_shift" for c in inputs[input_ref]]] = (
        df.groupby(groupby)[inputs[input_ref]]
         .transform(lambda x: x.rolling(rolling_period, min_periods=min_periods).mean().shift(shift))
    )

    return df

rolling_period = 4

rsh_yd_X = calculate_rolling_data(rsh_yd_X, ["season", "week", "player_id"], "rsh_yd", ["season", "player_id"], rolling_period=rolling_period)
rsh_td_X = calculate_rolling_data(rsh_td_X, ["season", "week", "player_id"], "rsh_td", ["season", "player_id"], rolling_period=rolling_period)
rc_yd_X = calculate_rolling_data(rc_yd_X, ["season", "week", "player_id"], "rc_yd", ["season", "player_id"], rolling_period=rolling_period)
rc_td_X = calculate_rolling_data(rc_td_X, ["season", "week", "player_id"], "rc_td", ["season", "player_id"], rolling_period=rolling_period)
rc_X = calculate_rolling_data(rc_X, ["season", "week", "player_id"], "rc", ["season", "player_id"], rolling_period=rolling_period)
p_yd_X = calculate_rolling_data(p_yd_X, ["season", "week", "player_id"], "p_yd", ["season", "player_id"], rolling_period=rolling_period)
p_td_X = calculate_rolling_data(p_td_X, ["season", "week", "player_id"], "p_td", ["season", "player_id"], rolling_period=rolling_period)
intcpt_X = calculate_rolling_data(intcpt_X, ["season", "week", "player_id"], "intcpt", ["season", "player_id"], rolling_period=rolling_period)
rsh_fmbls_X = calculate_rolling_data(rsh_fmbls_X, ["season", "week", "player_id"], "rsh_fmbls", ["season", "player_id"], rolling_period=rolling_period)
rc_fmbls_X = calculate_rolling_data(rc_fmbls_X, ["season", "week", "player_id"], "rc_fmbls", ["season", "player_id"], rolling_period=rolling_period)
defense_stats_df = calculate_rolling_data(defense_stats_df, ["season", "week", "team"], "def", ["season", "team"], rolling_period=rolling_period)

Normalize the data using Z-Score Standardization

In [12]:
# Normalize the data using Z-Score Standardization (sklearn StandardScaler)
from sklearn.preprocessing import StandardScaler

rsh_yd_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rsh_yd"]] + ["depth_team"] + list(encoded_feature_names) 
rsh_td_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rsh_td"]] + ["depth_team"] + list(encoded_feature_names)
rc_yd_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rc_yd"]] + ["depth_team"] + list(encoded_feature_names)
rc_td_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rc_td"]] + ["depth_team"] + list(encoded_feature_names)
rc_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rc"]] + ["depth_team"] + list(encoded_feature_names)
p_yd_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["p_yd"]] + ["depth_team"] + list(encoded_feature_names)
p_td_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["p_td"]] + ["depth_team"] + list(encoded_feature_names)
intcpt_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["intcpt"]] + ["depth_team"] + list(encoded_feature_names)
rsh_fmbls_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rsh_fmbls"]] + ["depth_team"] + list(encoded_feature_names)
rc_fmbls_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["rc_fmbls"]] + ["depth_team"] + list(encoded_feature_names)
def_input_cols = [col + f"_roll{rolling_period}_shift" for col in inputs["def"]]

scalers = {}

def scale_inplace(df, cols, name):
    # optionally: df = df.copy()  # if you want to avoid mutating the original
    scaler = StandardScaler()
    df.loc[:, cols] = scaler.fit_transform(df[cols])
    scalers[name] = scaler
    return df

rsh_yd_X = scale_inplace(rsh_yd_X, rsh_yd_input_cols, "rsh_yd")
rsh_td_X = scale_inplace(rsh_td_X, rsh_td_input_cols, "rsh_td")
rc_yd_X = scale_inplace(rc_yd_X,  rc_yd_input_cols,  "rc_yd")
rc_td_X = scale_inplace(rc_td_X,  rc_td_input_cols,  "rc_td")
p_yd_X = scale_inplace(p_yd_X,  p_yd_input_cols,  "p_yd")
p_td_X = scale_inplace(p_td_X,  p_td_input_cols,  "p_td")
intcpt_X = scale_inplace(intcpt_X,  intcpt_input_cols,  "intcpt")
rsh_fmbls_X = scale_inplace(rsh_fmbls_X, rsh_fmbls_input_cols, "rsh_fmbls")
rc_fmbls_X = scale_inplace(rc_fmbls_X, rc_fmbls_input_cols, "rc_fmbls")
defense_stats_df = scale_inplace(defense_stats_df, def_input_cols, "def")

Merge offense and defense

In [13]:
# Merge the defensive dataframe to the offensive one.
rsh_yd_X = rsh_yd_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rsh_td_X = rsh_td_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_yd_X = rc_yd_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_td_X = rc_td_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_X = rc_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
p_yd_X = p_yd_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
p_td_X = p_td_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
intcpt_X = intcpt_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rsh_fmbls_X = rsh_fmbls_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])
rc_fmbls_X = rc_fmbls_X.merge(defense_stats_df, how="left", left_on=["opponent_team", "season", "week"], right_on=["team", "season", "week"])

### Apply the Bayesian Model

In [14]:
import pymc as pm
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import r2_score

"""
Build the model to project player yards for rushing and receiving.
"""
# df: one row per (player, week)
# y = yards (float)
# X = scaled features (numpy array), shape (n, p)

def fit_bayesian_model(X, y):
    with pm.Model() as model:
        alpha = pm.Normal("alpha", 0, 1)
        beta = pm.Normal("beta", 0, 1, shape=X.shape[1])
        sigma = pm.HalfNormal("sigma", 1)

        mu = alpha + pm.math.dot(X, beta)
        y_obs = pm.Normal("y_obs", mu=mu, sigma=sigma, observed=y)

        idata = pm.sample()
        return model, y_obs, idata
    
def predict_from_fitted(idata, X_new, predictive=True, ci=(5, 95), seed=0):
    """
    Make predictions from your fitted Bayesian regression model.
    
    Parameters
    ----------
    idata : arviz.InferenceData
        The posterior samples returned by pm.sample().
    X_new : array-like of shape (n, p)
        New feature matrix (must be scaled and ordered like training X).
    predictive : bool
        If True, return posterior predictive (includes noise).
        If False, return posterior mean function (no added noise).
    ci : tuple
        Credible interval percentiles (default 90% CI).
    seed : int
        Random seed for reproducibility.
    """
    Xn = np.asarray(X_new, dtype=float)

    post = idata.posterior
    alpha = post["alpha"].values.reshape(-1)                  # (S,)
    beta  = post["beta"].values.reshape(-1, Xn.shape[1])      # (S, p)
    sigma = post["sigma"].values.reshape(-1)                  # (S,)

    # Compute linear mean function draws (S,n)
    mu_draws = beta @ Xn.T + alpha[:, None]

    if predictive:
        rng = np.random.default_rng(seed)
        y_draws = mu_draws + rng.normal(0.0, sigma[:, None], size=mu_draws.shape)
    else:
        y_draws = mu_draws

    # Summaries
    pred_mean   = y_draws.mean(axis=0)
    pred_median = np.median(y_draws, axis=0)
    lo, hi      = np.percentile(y_draws, ci, axis=0)

    return pred_mean, pred_median, (lo, hi), y_draws

model_results = {}
idatas = {}
models = {}

input_dataframes = [rsh_yd_X, rsh_td_X, rc_yd_X, rc_td_X, rc_X, p_yd_X, p_td_X, intcpt_X, rsh_fmbls_X, rc_fmbls_X]  
inputs = [rsh_yd_input_cols, rsh_td_input_cols, rc_yd_input_cols, rc_td_input_cols, rc_input_cols, p_yd_input_cols, p_td_input_cols, intcpt_input_cols, rsh_fmbls_input_cols, rc_fmbls_input_cols]
targets = ["rushing_yards", "rushing_tds", "receiving_yards", "receiving_tds", "receptions", "passing_yards", "passing_tds", "passing_interceptions", "rushing_fumbles_lost", "receiving_fumbles_lost"]

for df, input, target in zip(input_dataframes, inputs, targets):
    X = df.dropna()[input + def_input_cols].reset_index(drop=True)
    y = df.dropna()[target].reset_index(drop=True)

    X_train, X_valid, y_train, y_valid = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    model, y_obs, idata = fit_bayesian_model(X_train, y_train)

    models[target] = model 
    idatas[target] = idata

    pred_mean, pred_median, (pred_lo, pred_hi), y_draws = predict_from_fitted(idata, X)
 
    # using mean
    rmse_mean = root_mean_squared_error(y, pred_mean)
    r2_mean = r2_score(y, pred_mean)

    # using median
    rmse_median = root_mean_squared_error(y, pred_median)
    r2_median = r2_score(y, pred_median)    

    # using lo
    rmse_lo = root_mean_squared_error(y, pred_lo)
    r2_lo = r2_score(y, pred_lo)

    #using hi
    rmse_hi = root_mean_squared_error(y, pred_hi)  
    r2_hi = r2_score(y, pred_hi)

    model_results[target] = {"mean": {"rmse": rmse_mean, "r2": r2_mean}, "median": {"rmse": rmse_median, "r2": r2_median}, "low": {"rmse": rmse_lo, "r2": r2_lo}, "high": {"rmse": rmse_hi, "r2": r2_hi}}

Initializing NUTS using jitter+adapt_diag...
c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\pytensor\link\c\cmodule.py:2968: UserWarning: PyTensor could not link to a BLAS installation. Operations that might benefit from BLAS will be severely degraded.
This usually happens when PyTensor is installed via pip. We recommend it be installed via conda/mamba/pixi instead.
Alternatively, you can use an experimental backend such as Numba or JAX that perform their own BLAS optimizations, by setting `pytensor.config.mode == 'NUMBA'` or passing `mode='NUMBA'` when compiling a PyTensor function.
For more options and details see https://pytensor.readthedocs.io/en/latest/troubleshooting.html#how-do-i-configure-test-my-blas-library
  warnings.warn(
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, sigma]


c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 6796 seconds.
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta, sigma]


c:\Users\bengu\.virtualenvs\cfeproj-oIABPDjj\lib\site-packages\rich\live.py:256: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

ValueError: Not enough samples to build a trace.

In [ ]:
mo

In [15]:
print(model_results)

{'rushing_yards': {'mean': {'rmse': 17.619336047816436, 'r2': 0.5927269139353224}, 'median': {'rmse': 17.620359498406494, 'r2': 0.592679598192539}, 'low': {'rmse': 33.97647673357516, 'r2': -0.5144792147908259}, 'high': {'rmse': 33.88928591623462, 'r2': -0.5067162410598429}}}


### Check Sample Predictions

In [ ]:
# Params
SEASON = 2025
WEEK = 4

# Map dataframe names to their corresponding input-cols variable names
datasets = {
    "rsh_yd_X": "rsh_yd_input_cols",
    "rsh_td_X": "rsh_td_input_cols",
    "rc_yd_X":  "rc_yd_input_cols",
    "rc_td_X":  "rc_td_input_cols",
    "rc_X":     "rc_input_cols",
    "p_yd_X":   "p_yd_input_cols",
    "p_td_X":   "p_td_input_cols",
    "intcpt_X": "intcpt_input_cols",
    "rsh_fmbls_X": "rsh_fmbls_input_cols",
    "rc_fmbls_X":  "rc_fmbls_input_cols",
}

for df_name, cols_name in datasets.items():
    df = globals()[df_name]
    # 1) filter to current week
    cw_filtered = (
        df[(df["season"] == SEASON) & (df["week"] == WEEK)]
        .dropna()
        .reset_index(drop=True)
    )
    globals()[f"cw_{df_name}"] = cw_filtered


In [ ]:
from collections import defaultdict
input_dataframes = [cw_rsh_yd_X, cw_rsh_td_X, cw_rc_yd_X, cw_rc_td_X, cw_rc_X, cw_p_yd_X, cw_p_td_X, cw_intcpt_X, cw_rsh_fmbls_X, cw_rc_fmbls_X]
inputs = [rsh_yd_input_cols, rsh_td_input_cols, rc_yd_input_cols, rc_td_input_cols, rc_input_cols, p_yd_input_cols, p_td_input_cols, intcpt_input_cols, rsh_fmbls_input_cols, rc_fmbls_input_cols]
targets = ["rushing_yards", "rushing_tds", "receiving_yards", "receiving_tds", "receptions", "passing_yards", "passing_tds", "passing_interceptions", "rushing_fumbles_lost", "receiving_fumbles_lost"]

player_projections = defaultdict(dict)

for df, input, target in zip(input_dataframes, inputs, targets):
    X = df.dropna()[input + def_input_cols].reset_index(drop=True)
    y = df.dropna()[target].reset_index(drop=True)

    # Get the model from saved
    reg = models[target]

    # Evaluate
    preds = reg.predict(X)

    for i, pred in enumerate(preds):
        player_id, player_name = df.loc[i, "player_id"], df.loc[i, "player_name"]
        if player_id not in player_projections:
            player_projections[player_id]["player_name"] = player_name
        
        player_data = POSITION_PLAYER_STAT_PROJECTION_DATA_DICT[target].copy()
        player_data["projection"] = pred
        player_data["true"] = y.iloc[i]

        player_projections[player_id][target] = player_data

RMSE Mean: 19.53405859471916
R2 Mean: 0.5791772442435261
RMSE Median: 19.53689934109476
R2 Median: 0.5790548387908364
RMSE Lo: 37.581563136541014
R2 Lo: -0.5576286383380709
RMSE Hi: 37.47903163607568
R2 Hi: -0.5491410646426385


In [ ]:
week_4_projections = []
 
for player_id, data in player_projections.items():
    projected_points = 0
    true_points = 0
    for target in targets:
        if target in data:
            projected_points += data[target]["projection"] * data[target]["weight"]
            true_points += data[target]["true"] * data[target]["weight"]

    week_4_projections.append((data["player_name"], projected_points, true_points))

In [ ]:
true = [x[2] for x in week_4_projections]
preds = [x[1] for x in week_4_projections]

error = root_mean_squared_error(true, preds)
r2 = r2_score(true, preds)
print(error, r2)